In [71]:
import pandas as pd
import numpy as np
import importlib
import helper
importlib.reload(helper)

<module 'helper' from '/Users/home/Documents/Projects/ai-engineering-portfolio/classical-ml/house-price-prediction/notebooks/helper.py'>

In [72]:
train_df = pd.read_csv("../data/raw/train.csv")
test_df = pd.read_csv("../data/raw/test.csv")

In [73]:
train_percent_nan = helper.percent_missing(train_df)
train_percent_nan

Electrical       0.068493
MasVnrArea       0.547945
BsmtQual         2.534247
BsmtCond         2.534247
BsmtFinType1     2.534247
BsmtExposure     2.602740
BsmtFinType2     2.602740
GarageCond       5.547945
GarageQual       5.547945
GarageFinish     5.547945
GarageYrBlt      5.547945
GarageType       5.547945
LotFrontage     17.739726
FireplaceQu     47.260274
MasVnrType      59.726027
Fence           80.753425
Alley           93.767123
MiscFeature     96.301370
PoolQC          99.520548
dtype: float64

In [74]:
test_percent_nan = helper.percent_missing(test_df)
test_percent_nan

TotalBsmtSF      0.068540
GarageArea       0.068540
GarageCars       0.068540
KitchenQual      0.068540
BsmtUnfSF        0.068540
BsmtFinSF2       0.068540
BsmtFinSF1       0.068540
SaleType         0.068540
Exterior1st      0.068540
Exterior2nd      0.068540
Functional       0.137080
Utilities        0.137080
BsmtHalfBath     0.137080
BsmtFullBath     0.137080
MSZoning         0.274160
MasVnrArea       1.028101
BsmtFinType1     2.878684
BsmtFinType2     2.878684
BsmtQual         3.015764
BsmtExposure     3.015764
BsmtCond         3.084304
GarageType       5.209047
GarageCond       5.346127
GarageQual       5.346127
GarageYrBlt      5.346127
GarageFinish     5.346127
LotFrontage     15.558602
FireplaceQu     50.034270
MasVnrType      61.274846
Fence           80.123372
Alley           92.666210
MiscFeature     96.504455
PoolQC          99.794380
dtype: float64

In [75]:
## drop id columns
train_df = train_df.drop(columns='Id',axis=1)
test_df = test_df.drop(columns='Id',axis=1)

In [76]:
##drop columns with more than 80% row missing

train_df = train_df.drop(columns = ['Fence','Alley','MiscFeature','PoolQC'],axis=1)
test_df = test_df.drop(columns = ['Fence','Alley','MiscFeature','PoolQC'],axis=1)


In [77]:
train_percent_nan[train_percent_nan<1]

Electrical    0.068493
MasVnrArea    0.547945
dtype: float64

In [78]:
train_df[train_df['Electrical'].isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1379,80,RL,73.0,9735,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,5,2008,WD,Normal,167500


In [79]:
## Electrical has only one row missing so filling it with most common value i.e SBrkr
train_df['Electrical'] = train_df['Electrical'].fillna('SBrkr')
train_df['Electrical'].value_counts()

Electrical
SBrkr    1335
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: count, dtype: int64

In [80]:
train_df[train_df['BsmtQual'].isnull()][['BsmtQual','BsmtCond','BsmtFinType1','BsmtExposure','BsmtFinType2']]

,BsmtQual,BsmtCond,BsmtFinType1,BsmtExposure,BsmtFinType2
17,NaN,NaN,NaN,NaN,NaN
39,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN
156,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN
259,NaN,NaN,NaN,NaN,NaN
342,NaN,NaN,NaN,NaN,NaN
362,NaN,NaN,NaN,NaN,NaN
371,NaN,NaN,NaN,NaN,NaN


In [82]:
##we can assume that these data dont have basement and hence fill with appropriate no basement data
bsmt_nan_col = ['BsmtQual','BsmtCond','BsmtFinType1','BsmtExposure','BsmtFinType2']
train_df[bsmt_nan_col] = train_df[bsmt_nan_col].fillna('NA')

In [83]:
train_percent_nan = helper.percent_missing(train_df)
train_percent_nan

MasVnrArea       0.547945
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageQual       5.547945
GarageCond       5.547945
LotFrontage     17.739726
FireplaceQu     47.260274
MasVnrType      59.726027
dtype: float64

In [89]:
train_df[train_df['GarageType'].isnull()][['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 39 to 1453
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   GarageType    0 non-null      object 
 1   GarageYrBlt   0 non-null      float64
 2   GarageFinish  0 non-null      object 
 3   GarageQual    0 non-null      object 
 4   GarageCond    0 non-null      object 
dtypes: float64(1), object(4)
memory usage: 3.8+ KB


In [ ]:
##fill missing garage data rows assuming no garage
train_df['GarageYrBlt']=train_df['GarageYrBlt'].fillna(0)
garage_fill_col = ['GarageType','GarageFinish','GarageQual','GarageCond']
train_df[garage_fill_col] = train_df[garage_fill_col].fillna('NA')

In [91]:
train_percent_nan = helper.percent_missing(train_df)
train_percent_nan

MasVnrArea      0.547945
LotFrontage    17.739726
FireplaceQu    47.260274
MasVnrType     59.726027
dtype: float64

In [95]:
train_df[train_df['MasVnrType'].isnull()][['MasVnrArea']].value_counts()

MasVnrArea
0.0           859
1.0             2
288.0           1
312.0           1
344.0           1
Name: count, dtype: int64

In [97]:
##fill MasVnrtype and MasVnrArea with None and 0
train_df['MasVnrType'] = train_df['MasVnrType'].fillna('None')
train_df['MasVnrArea'] = train_df['MasVnrArea'].fillna(0)

In [98]:
train_percent_nan = helper.percent_missing(train_df)
train_percent_nan

LotFrontage    17.739726
FireplaceQu    47.260274
dtype: float64

In [100]:
train_df[train_df['FireplaceQu'].isnull()][['Fireplaces']].value_counts()

Fireplaces
0             690
Name: count, dtype: int64

In [101]:
## fill FireplaceQu wit NA as Fireplaces field is having 0
train_df['FireplaceQu'] = train_df['FireplaceQu'].fillna(0)

In [107]:
train_df.groupby('Neighborhood')['LotFrontage'].transform(lambda value : value.fillna(value.mean))

mean_LotFrontage_by_Neighborhood = train_df.groupby('Neighborhood')['LotFrontage'].mean()
mean_LotFrontage_by_Neighborhood

Neighborhood
Blmngtn    47.142857
Blueste    24.000000
BrDale     21.562500
BrkSide    57.509804
ClearCr    83.461538
CollgCr    71.682540
Crawfor    71.804878
Edwards    68.217391
Gilbert    79.877551
IDOTRR     62.500000
MeadowV    27.800000
Mitchel    70.083333
NAmes      76.462366
NPkVill    32.285714
NWAmes     81.288889
NoRidge    91.878788
NridgHt    81.881579
OldTown    62.788991
SWISU      58.913043
Sawyer     74.437500
SawyerW    71.500000
Somerst    64.666667
StoneBr    62.700000
Timber     80.133333
Veenker    59.714286
Name: LotFrontage, dtype: float64

In [110]:
##based on domain knowledge which suggest LotFrontage i.e Linear feet of street connected to property is closly determined by the neighborhood , 
# fill missing LotFrontage with mean of neighborhood
train_df['LotFrontage'] = train_df['LotFrontage'].fillna(train_df['Neighborhood'].map(mean_LotFrontage_by_Neighborhood))

In [111]:
train_percent_nan = helper.percent_missing(train_df)
train_percent_nan

Series([], dtype: float64)